## Введение

Работа по освоению линейной классификатор. Для этого считаю необходимым написать базовый алгоритм самостоятельно, а потом сравнить его с готовым решением из пакета sklearn. Для работы буду использовать модель линейной классификации с L2 регуляризацией и стохастический градиентный спуск как оптимизатор.

In [1]:
from torch.nn import Linear
import torch
import numpy as np
from sklearn.datasets import load_breast_cancer
import torch.nn.functional as F
from sklearn.model_selection import train_test_split


In [2]:
iris = load_breast_cancer()

In [3]:
# iris.target = np.where(iris.target == 0, -1,iris.target)

In [4]:
train_X,test_X,train_Y,test_Y = train_test_split(iris.data,
                                                iris.target,
                                                test_size = 0.33,
                                                random_state = 42)

train_X_torch = torch.tensor(train_X, dtype = torch.float64)
train_Y_torch = torch.tensor(train_Y, dtype = torch.float64)
test_X_torch = torch.tensor(test_X, dtype = torch.float64)
test_Y_torch = torch.tensor(test_Y, dtype = torch.float64)

In [5]:
class LinearClassifier():
    def __init__(self,n_features,max_iter = 1000):
        self.n_features = n_features
        self.max_iter = max_iter
        self.model = torch.nn.Sequential()
        
        self.model.add_module('first', torch.nn.Linear(self.n_features,1))
        self.model.add_module('second', torch.nn.Sigmoid())
        self.model =  self.model.double()
        self.optimizer = torch.optim.Adam(self.model.parameters(), lr = 1e-3)
    
    def fit(self,design_matrix,target):
        history = []
        for i in range(self.max_iter):
            sampling  = np.random.randint(0,design_matrix.shape[1],30)
            part_design_matrix = design_matrix[sampling]
            part_target_vector = target[sampling]
            self.wight = torch.tensor(self.model.state_dict()['first.weight'],
                                      requires_grad=True,
                                      dtype = torch.float64
                                     )
            self.bias = torch.tensor(self.model.state_dict()['first.bias'],
                                     requires_grad=True,
                                     dtype = torch.float64
                                    )
            loss = torch.mean(
               torch.log(1+torch.exp(-(
                    part_target_vector*(
                        part_design_matrix.matmul(self.wight.transpose(0,1)))))))
#             pred = self.model(part_design_matrix)[:,0]
#             loss = torch.mean(F.binary_cross_entropy(pred,
#                                                     part_target_vector,
#                                                     reduce = 'none'),dim = 0,keepdim = True)



            loss.backward()
            self.optimizer.step()
            self.optimizer.zero_grad()
            history.append(loss.data.numpy())
            if i % 100 == 0:
                print('loss{}:{}'.format(i, np.mean(history[-100:])))

        return self.model
    
    def predict(self,design_matrix):
        return self.model(design_matrix)

            

In [6]:
torch_classifier = LinearClassifier(30)

In [7]:
torch_classifier.fit(train_X_torch,train_Y_torch)

C:\Users\Anton\Anaconda3\lib\site-packages\ipykernel_launcher.py:20: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
C:\Users\Anton\Anaconda3\lib\site-packages\ipykernel_launcher.py:24: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


loss0:1.2951315960983258
loss100:2.63324541956442
loss200:2.6997578136708076
loss300:2.9933239460377656
loss400:2.844085392103214
loss500:2.7796955514354322
loss600:2.655453630302964
loss700:2.934714461174349
loss800:2.856090234855324
loss900:2.794499919727765


Sequential(
  (first): Linear(in_features=30, out_features=1, bias=True)
  (second): Sigmoid()
)

In [8]:
final_pred = torch_classifier.predict(test_X_torch)[:,0]


In [9]:
final_pred

tensor([9.9834e-01, 1.0176e-10, 4.9593e-03, 1.0000e+00, 1.0000e+00, 6.6563e-01,
        3.4365e-03, 1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00, 9.9742e-01,
        1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00,
        9.9842e-01, 9.9988e-01, 9.9915e-01, 1.0000e+00, 8.6419e-05, 9.9990e-01,
        1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00, 7.2859e-17,
        1.0000e+00, 1.0000e+00, 1.0000e+00, 9.9980e-01, 1.0000e+00, 1.0000e+00,
        3.4976e-01, 1.0000e+00, 2.7634e-06, 1.0000e+00, 1.0000e+00, 9.1572e-01,
        1.0000e+00, 9.9998e-01, 1.0000e+00, 9.9969e-01, 9.9999e-01, 9.9994e-01,
        9.9954e-01, 1.0000e+00, 5.0370e-06, 2.6055e-03, 1.0000e+00, 1.0000e+00,
        1.0000e+00, 9.9999e-01, 1.0000e+00, 9.9999e-01, 9.9993e-01, 1.0000e+00,
        1.0000e+00, 9.9566e-01, 1.0000e+00, 1.0000e+00, 1.0000e+00, 9.9998e-01,
        2.4779e-06, 1.0000e+00, 1.0000e+00, 1.0000e+00, 9.9995e-01, 1.4698e-07,
        1.0000e+00, 1.0000e+00, 9.7084e-

In [10]:
final_pred = np.array(final_pred > 0.6)

In [11]:
final_pred

array([ True, False, False,  True,  True,  True, False,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True, False,  True,  True,  True,  True,
        True,  True, False,  True,  True,  True,  True,  True,  True,
       False,  True, False,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True, False, False,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True, False,  True,  True,  True,  True, False,
        True,  True,  True,  True,  True,  True,  True,  True, False,
        True,  True,  True,  True,  True, False,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True, False,  True,
        True, False,  True, False,  True,  True,  True,  True, False,
        True,  True,  True,  True,  True, False,  True, False,  True,
        True,  True,  True,  True,  True,  True,  True,  True, False,
       False,  True,

In [12]:
np.mean(np.array(final_pred) == np.array(test_Y_torch))*100

81.38297872340425